<a href="https://colab.research.google.com/github/yumpy/fastai-course-v3/blob/master/notes/dl1/pets-resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>